In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
URL = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('select', class_='SelectNav__select')

In [3]:
list_url_journey = []
for result in results:
    liens_journey = result.find_all('option')
    for liens in liens_journey:
        list_url_journey.append(liens.get('value'))

In [4]:
row_list = []
for url_journey in list_url_journey:
    journey = pd.read_json(url_journey, lines=True)
    for match in journey['items'][0]:
        try:
            if len(match['items']) == 1:
                match_dict={}
                match_dict['place'] = match['items'][0]['event']['lieu']['ville']
                match_dict['date'] = match['items'][0]['event']['date']
                match_dict['dom_name'] = match['items'][0]['event']['specifics']['domicile']['equipe']['nom']
                match_dict['ext_name'] = match['items'][0]['event']['specifics']['exterieur']['equipe']['nom']
                match_dict['but_dom'] = match['items'][0]['event']['specifics']['score']['domicile']
                match_dict['but_ext'] = match['items'][0]['event']['specifics']['score']['exterieur']
                match_dict['winner'] = match['items'][0]['event']['specifics']['vainqueur']
                match_dict['lien'] = match['items'][0]['event']['lien_web']
                row_list.append(match_dict)
            elif len(match['items']) > 1:
                for m in range(len(match['items'])):
                    match_dict={}
                    match_dict['place'] = match['items'][m]['event']['lieu']['ville']
                    match_dict['date'] = match['items'][m]['event']['date']
                    match_dict['dom_name'] = match['items'][m]['event']['specifics']['domicile']['equipe']['nom']
                    match_dict['ext_name'] = match['items'][m]['event']['specifics']['exterieur']['equipe']['nom']
                    match_dict['but_dom'] = match['items'][m]['event']['specifics']['score']['domicile']
                    match_dict['but_ext'] = match['items'][m]['event']['specifics']['score']['exterieur']
                    match_dict['winner'] = match['items'][m]['event']['specifics']['vainqueur']
                    match_dict['lien'] = match['items'][m]['event']['lien_web']
                    row_list.append(match_dict)
        except:
            pass

data = pd.DataFrame(row_list)

In [5]:
data.shape

(380, 8)

In [6]:
data.head()

,place,date,dom_name,ext_name,but_dom,but_ext,winner,lien
0,Bordeaux,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,0,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
1,Dijon,2020-08-22T17:00:00+0200,Dijon,Angers,0,1,exterieur,https://www.lequipe.fr/Football/match-direct/l...
2,Lille,2020-08-22T21:00:00+0200,Lille,Rennes,1,1,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
3,Monaco,2020-08-23T13:00:00+0200,Monaco,Reims,2,2,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...
4,Lorient,2020-08-23T15:00:00+0200,Lorient,Strasbourg,3,1,domicile,https://www.lequipe.fr/Football/match-direct/l...


In [7]:
df_matchs = data[['place', 'date']]
df_matchs

,place,date
0,Bordeaux,2020-08-21T19:00:00+0200
1,Dijon,2020-08-22T17:00:00+0200
2,Lille,2020-08-22T21:00:00+0200
3,Monaco,2020-08-23T13:00:00+0200
4,Lorient,2020-08-23T15:00:00+0200
...,...,...
375,Rennes,2021-05-23T21:00:00+0200
376,Nantes,2021-05-23T21:00:00+0200
377,Saint-Étienne,2021-05-23T21:00:00+0200
378,Reims,2021-05-23T21:00:00+0200


In [9]:
list_liens_matchs = data['lien'].tolist()
#list_liens_matchs

In [45]:
points_dict_dom={'exterieur':0, 'ex-aequo':1,'domicile':3}

df_matchs_dom = data[['place','date','dom_name','but_dom','winner']]
df_matchs_dom.insert(3,'home', True)
df_matchs_dom['winner'] = df_matchs_dom['winner'].map(points_dict_dom)
df_matchs_dom.rename(columns={'dom_name':'team','but_dom':'team_goals','winner':'points'}, inplace=True)
df_matchs_dom

<ipython-input-45-fd55b1dd00e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matchs_dom['winner'] = df_matchs_dom['winner'].map(points_dict_dom)
/home/juliecharrier/anaconda3/envs/ecole_ia/lib/python3.9/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,place,date,team,home,team_goals,points
0,Bordeaux,2020-08-21T19:00:00+0200,Bordeaux,True,0,1
1,Dijon,2020-08-22T17:00:00+0200,Dijon,True,0,0
2,Lille,2020-08-22T21:00:00+0200,Lille,True,1,1
3,Monaco,2020-08-23T13:00:00+0200,Monaco,True,2,1
4,Lorient,2020-08-23T15:00:00+0200,Lorient,True,3,3
...,...,...,...,...,...,...
375,Rennes,2021-05-23T21:00:00+0200,Rennes,True,2,3
376,Nantes,2021-05-23T21:00:00+0200,Nantes,True,1,0
377,Saint-Étienne,2021-05-23T21:00:00+0200,Saint-Étienne,True,0,0
378,Reims,2021-05-23T21:00:00+0200,Reims,True,1,0


In [46]:
points_dict_ext={'exterieur':3, 'ex-aequo':1,'domicile':0}

df_matchs_ext = data[['place', 'date', 'ext_name', 'but_ext', 'winner']]
df_matchs_ext.insert(3, 'home', False)
df_matchs_ext['winner'] = df_matchs_ext['winner'].map(points_dict_ext)
df_matchs_ext.rename(columns={'ext_name': 'team', 'but_ext': 'team_goals', 'winner': 'points'}, inplace=True)
df_matchs_ext


<ipython-input-46-d4f71e3a0d8a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matchs_ext['winner'] = df_matchs_ext['winner'].map(points_dict_ext)
/home/juliecharrier/anaconda3/envs/ecole_ia/lib/python3.9/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,place,date,team,home,team_goals,points
0,Bordeaux,2020-08-21T19:00:00+0200,Nantes,False,0,1
1,Dijon,2020-08-22T17:00:00+0200,Angers,False,1,3
2,Lille,2020-08-22T21:00:00+0200,Rennes,False,1,1
3,Monaco,2020-08-23T13:00:00+0200,Reims,False,2,1
4,Lorient,2020-08-23T15:00:00+0200,Strasbourg,False,1,0
...,...,...,...,...,...,...
375,Rennes,2021-05-23T21:00:00+0200,Nîmes,False,0,0
376,Nantes,2021-05-23T21:00:00+0200,Montpellier,False,2,3
377,Saint-Étienne,2021-05-23T21:00:00+0200,Dijon,False,1,3
378,Reims,2021-05-23T21:00:00+0200,Bordeaux,False,2,3


In [82]:
df_matchs_total = pd.concat([df_matchs_dom, df_matchs_ext], ignore_index=True)
df_matchs_total

,place,date,team,home,team_goals,points
0,Bordeaux,2020-08-21T19:00:00+0200,Bordeaux,True,0,1
1,Dijon,2020-08-22T17:00:00+0200,Dijon,True,0,0
2,Lille,2020-08-22T21:00:00+0200,Lille,True,1,1
3,Monaco,2020-08-23T13:00:00+0200,Monaco,True,2,1
4,Lorient,2020-08-23T15:00:00+0200,Lorient,True,3,3
...,...,...,...,...,...,...
755,Rennes,2021-05-23T21:00:00+0200,Nîmes,False,0,0
756,Nantes,2021-05-23T21:00:00+0200,Montpellier,False,2,3
757,Saint-Étienne,2021-05-23T21:00:00+0200,Dijon,False,1,3
758,Reims,2021-05-23T21:00:00+0200,Bordeaux,False,2,3


In [83]:

df_matchs_total['date'] = pd.to_datetime(df_matchs_total['date'], format="%Y-%m-%dT%H:%M:%S%z")
df_matchs_total['date']


0      2020-08-21 19:00:00+02:00
1      2020-08-22 17:00:00+02:00
2      2020-08-22 21:00:00+02:00
3      2020-08-23 13:00:00+02:00
4      2020-08-23 15:00:00+02:00
                 ...            
755    2021-05-23 21:00:00+02:00
756    2021-05-23 21:00:00+02:00
757    2021-05-23 21:00:00+02:00
758    2021-05-23 21:00:00+02:00
759    2021-05-23 21:00:00+02:00
Name: date, Length: 760, dtype: object

In [84]:
df_matchs_total['team'] = df_matchs_total['team'].replace({'Paris-SG':'Paris'})
df_matchs_total['team'].unique()

array(['Bordeaux', 'Dijon', 'Lille', 'Monaco', 'Lorient', 'Nîmes', 'Nice',
       'Montpellier', 'Paris', 'Marseille', 'Lyon', 'Rennes',
       'Strasbourg', 'Reims', 'Nantes', 'Saint-Étienne', 'Angers', 'Metz',
       'Brest', 'Lens'], dtype=object)

In [85]:
df = pd.DataFrame()
df['match'] = df_matchs_total['date'].astype(str) + ' ' + df_matchs_total['place']
df_matchs_total.insert(0,'match',df['match'])
df_matchs_total

,match,place,date,team,home,team_goals,points
0,2020-08-21 19:00:00+02:00 Bordeaux,Bordeaux,2020-08-21 19:00:00+02:00,Bordeaux,True,0,1
1,2020-08-22 17:00:00+02:00 Dijon,Dijon,2020-08-22 17:00:00+02:00,Dijon,True,0,0
2,2020-08-22 21:00:00+02:00 Lille,Lille,2020-08-22 21:00:00+02:00,Lille,True,1,1
3,2020-08-23 13:00:00+02:00 Monaco,Monaco,2020-08-23 13:00:00+02:00,Monaco,True,2,1
4,2020-08-23 15:00:00+02:00 Lorient,Lorient,2020-08-23 15:00:00+02:00,Lorient,True,3,3
...,...,...,...,...,...,...,...
755,2021-05-23 21:00:00+02:00 Rennes,Rennes,2021-05-23 21:00:00+02:00,Nîmes,False,0,0
756,2021-05-23 21:00:00+02:00 Nantes,Nantes,2021-05-23 21:00:00+02:00,Montpellier,False,2,3
757,2021-05-23 21:00:00+02:00 Saint-Étienne,Saint-Étienne,2021-05-23 21:00:00+02:00,Dijon,False,1,3
758,2021-05-23 21:00:00+02:00 Reims,Reims,2021-05-23 21:00:00+02:00,Bordeaux,False,2,3


In [86]:
df_matchs_total.drop(['place','date'], axis=1, inplace=True)
df_matchs_total.rename(columns={'team': 'team_id', 'match': 'match_id'}, inplace=True)
df_matchs_total

,match_id,team_id,home,team_goals,points
0,2020-08-21 19:00:00+02:00 Bordeaux,Bordeaux,True,0,1
1,2020-08-22 17:00:00+02:00 Dijon,Dijon,True,0,0
2,2020-08-22 21:00:00+02:00 Lille,Lille,True,1,1
3,2020-08-23 13:00:00+02:00 Monaco,Monaco,True,2,1
4,2020-08-23 15:00:00+02:00 Lorient,Lorient,True,3,3
...,...,...,...,...,...
755,2021-05-23 21:00:00+02:00 Rennes,Nîmes,False,0,0
756,2021-05-23 21:00:00+02:00 Nantes,Montpellier,False,2,3
757,2021-05-23 21:00:00+02:00 Saint-Étienne,Dijon,False,1,3
758,2021-05-23 21:00:00+02:00 Reims,Bordeaux,False,2,3


In [87]:
df_matchs_total['created_at'] = datetime.now()
df_matchs_total['updated_at'] = datetime.now()
df_matchs_total

,match_id,team_id,home,team_goals,points,created_at,updated_at
0,2020-08-21 19:00:00+02:00 Bordeaux,Bordeaux,True,0,1,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
1,2020-08-22 17:00:00+02:00 Dijon,Dijon,True,0,0,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
2,2020-08-22 21:00:00+02:00 Lille,Lille,True,1,1,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
3,2020-08-23 13:00:00+02:00 Monaco,Monaco,True,2,1,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
4,2020-08-23 15:00:00+02:00 Lorient,Lorient,True,3,3,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
...,...,...,...,...,...,...,...
755,2021-05-23 21:00:00+02:00 Rennes,Nîmes,False,0,0,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
756,2021-05-23 21:00:00+02:00 Nantes,Montpellier,False,2,3,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
757,2021-05-23 21:00:00+02:00 Saint-Étienne,Dijon,False,1,3,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565
758,2021-05-23 21:00:00+02:00 Reims,Bordeaux,False,2,3,2021-07-02 11:07:34.093920,2021-07-02 11:07:34.571565


In [88]:
df_matchs_total.to_csv('teams_matches_table.csv')